Time to optimize our function. Will be trying some methods.

In [13]:
import math
import scipy as sp
from scipy.optimize import basinhopping
from scipy.optimize import dual_annealing
import numpy as np


class DataSet:
    time = []
    stress = []
    strain_rate = 0
    def __init__(self,t,y,strate):
        self.time = t
        self.stress = y
        self.strain_rate = strate
    
    def getTime(self):
        return self.time
    
    def getStress(self):
#         print(self.time)
        return self.stress
    
    def getStrRate(self):
        return self.strain_rate

bounds =[(0.000001,1),(0.000001,1),(0,10000000),(0,10000000),(0,10000000),(0,10000000)]


t = [0.00000000001,0.15,0.3,0.45,0.6,0.75,0.9,1.05,1.2,1.35,1.5,1.65,1.8,1.95,2.1,2.25,2.4,2.55]
st = [0,1.789052069,2.6435247,3.204272363,3.417890521,3.711615487,3.925233645,4.058744993,4.272363151,4.566088117,4.779706275,5.046728972,5.26034713,5.393858478,5.527369826,5.714285714,5.874499332,6.08811749]
data_200_mm = DataSet(t, st, 0.00333333333)

t2 = [0.000000000001,0.06,0.12,0.18,0.24,0.3,0.36,0.42,0.48,0.54,0.6,0.66,0.72,0.78,0.84,0.9,0.96,1.02,1.08,1.14,1.2]
st2 = [0,2.403204272,3.871829105,4.993324433,5.740987984,6.461949266,7.156208278,7.716955941,8.117489987,8.411214953,8.758344459,9.052069426,9.319092123,9.666221629,10.09345794,10.52069426,10.78771696,11.02803738,11.26835781,11.53538051,11.64218959]
data_500_mm = DataSet(t2, st2, 0.00833333333)

t3 = [0.000000000001,0.03,0.06,0.09,0.12,0.15,0.18,0.21,0.24,0.27,0.3,0.33,0.36,0.39,0.42,0.45,0.48,0.51,0.54,0.57]
st3 = [0,3.177570093,6.595460614,9.158878505,10.94793057,12.25634179,13.24432577,13.99198932,14.87316422,15.99465955,16.39519359,16.68891856,17.0894526,17.24966622,17.67690254,17.9706275,18.37116155,18.69158879,18.87850467,19.19893191]
data_1000_mm = DataSet(t3, st3, 0.0166666667)

def equation(x,a,b,e1,n1,e2,n2):
    e0 = data_1000_mm.getStrRate()
    
    deg1 = 1-a
    denom1 = deg1*sp.special.gamma(deg1)
#     print(deg1)
    ele_1 = x**deg1
    ele_1 = ele_1/denom1
    ele_1 *= e0
    ele_1 *= e1**deg1
    ele_1 *= n1**a
    
    deg2 = 1-b
    denom2 = deg2*sp.special.gamma(deg2)
#     print(deg2)

    ele_2 = x**deg2
    ele_2 = ele_2/denom2
    ele_2 *= e0
    ele_2 *= e2**deg2
    ele_2 *= n2**b
    return ele_1 + ele_2

def error_func(x):
    total = 0
    sec = data_1000_mm.getTime()
    newt = data_1000_mm.getStress()
    for ind in range(len(sec)):
        err = newt[ind] - equation(sec[ind],x[0],x[1],x[2],x[3],x[4],x[5])
        err = err**2
        total += err
    total = total/len(sec)
#     print(np.sqrt(total))
    return np.sqrt(total)


x0 = np.array([0.5,0.5,1,1,1,1])

# ret = basinhopping(error_func,x0,minimizer_kwargs={"method":'BFGS'},disp=True)
ret = dual_annealing(error_func,bounds,x0=x0,maxiter=20000000,seed=42)

sol = ret['x']
print(error_func(sol)) 
        

C:\Users\desktop\AppData\Local\Temp\ipykernel_21576\3399759832.py:46: RuntimeWarning: invalid value encountered in double_scalars
  denom1 = deg1*sp.special.gamma(deg1)
C:\Users\desktop\AppData\Local\Temp\ipykernel_21576\3399759832.py:55: RuntimeWarning: invalid value encountered in double_scalars
  denom2 = deg2*sp.special.gamma(deg2)


0.9163584045176214


In [14]:
for x in data_1000_mm.getTime():
    print(equation(x,*ret['x']))

0.00024836848218911725
5.903741438330886
7.886058782131342
9.34133427903743
10.533984891589919
11.562974653459749
12.477902699081403
13.307718026490296
14.071013260473633
14.780546713338135
15.445510066114734
16.072777827947178
16.6676463037127
17.23429553888175
17.776091265033735
18.295789649036912
18.795680477432423
19.277689948081335
19.74345615081287
20.19438559543848


In [7]:
print(*ret['x'])

0.6420268969225622 1e-06 2606.0349465526897 607.1958197370183 79.68463819811431 48378.89169538372


In [8]:
print(error_func(sol))

0.08779074607631453


0.00019383430438974945
1.4146187661035667
1.8244581980220091
2.1204110495798756
2.3611670406873433
2.5681492045413443
2.751916099283581
2.918551875089472
3.0719235312587165
3.214661287059582
3.3486456117497685
3.475273426530695
3.5956144431802426
3.7105082861775434
3.8206276190049637
3.926520730963317
4.028641178726128
4.127368973304954
4.223026074768274
4.31588795238234
4.4061923618797785


In [ ]:
#ok wtv now relaxation model or something idk.
rel_t = [0.000000001,0.2,0.4,0.6,0.8,1,1.2]
rel_y = [5.953177258,2.073578595,0.7023411371,0.3344481605,0.1337792642,0.03344481605,0]
data_200mm_relax = DataSet(rel_t,rel_y,rel_y[0])
rel_t2 = [0.000000001,0.2,0.4,0.6,0.8,1,1.2]
rel_y2 = [11.60535117,3.110367893,1.337792642,0.6688963211,0.3010033445,0.06688963211,0]
data_500mm_relax = DataSet(rel_t2,rel_y2,rel_y2[0])
rel_t3 = [0,0.2,0.4,0.6,0.8,1,1.2,1.4]
